In [2]:
from deps.dobot_api import DobotApiDashboard, DobotApiMove
from deps import utils
from deps import cv_core
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pynput import keyboard
import time

cont = cv_core.Contours()

In [2]:
dash = DobotApiDashboard('192.168.1.6', 29999)
move = DobotApiMove('192.168.1.6', 30003)
keys = utils.Keyboard(dash)

In [3]:
dash.ClearError()
dash.EnableRobot()

'0,{},EnableRobot();'

In [33]:
utils.report_mode(dash)

Mode: ROBOT_MODE_ENABLE


In [53]:
dash.SetCollisionLevel(1)

'0,{},SetCollisionLevel(1);'

In [57]:
dash.DisableRobot()

'0,{},DisableRobot();'

In [55]:
utils.get_pose(dash)

X = 321.505463
Y = 0.0
Z = -34.254505
r = 22.918932


array([321.505463,   0.      , -34.254505,  22.918932])

In [54]:
move.MovL(321.505454, 0, -50, 22.918932)

'0,{},MovL(321.505454,0.000000,-50.000000,22.918932);'

In [4]:
keys.execute()

Position saved!
Position saved!
Position saved!
Position saved!
Special key pressed: Key.esc


In [5]:
print("Detecting Anchors")
while True:
    cap = cv2.VideoCapture(0)
    cap = cv_core.set_res(cap, (1600, 1200))
    centers = cont.wait_for_anchors(cap, show = True)
    cap.release()
    if len(centers) == 4:
        break


xys = [(arr[0], arr[1]) for arr in keys.coords]
pix_coor = utils.assign_corners(centers)
robot_coor = utils.assign_corners(xys, reverse=True)


features_mm_to_pixels_dict = {}
for key, value in robot_coor.items():
    features_mm_to_pixels_dict[value] = pix_coor[key]


tf_mtx = cv_core.compute_tf_mtx(features_mm_to_pixels_dict)

Detecting Anchors


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/dobot/anaconda3/lib/python3.9/site-packages/cv2/qt/plugins"
QObject::moveToThread: Current thread (0x55b9b2173d10) is not the object's thread (0x55b9b2ab9760).
Cannot move to target thread (0x55b9b2173d10)

QObject::moveToThread: Current thread (0x55b9b2173d10) is not the object's thread (0x55b9b2ab9760).
Cannot move to target thread (0x55b9b2173d10)

Qt: Session management error: Could not open network socket
QObject::moveToThread: Current thread (0x55b9b2173d10) is not the object's thread (0x55b9b2ab9760).
Cannot move to target thread (0x55b9b2173d10)

QObject::moveToThread: Current thread (0x55b9b2173d10) is not the object's thread (0x55b9b2ab9760).
Cannot move to target thread (0x55b9b2173d10)

QObject::moveToThread: Current thread (0x55b9b2173d10) is not the object's thread (0x55b9b2ab9760).
Cannot move to target thread (0x55b9b2173d10)

QObject::moveToThread: Current thread (0x55b9b2173d10) is not the object

In [6]:
centers

[(94, 1118), (1488, 1108), (1491, 100), (90, 81)]

In [4]:
def on_change(val): pass

offset = 110
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, cv_core.camera_res_dict['1200'])
#out = cv2.VideoWriter('out.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 25, (640,480))
cont = cv_core.Contours()

cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')

cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', 1348, 1011)
cv2.createTrackbar('Manual Lock', 'frame', 0, 1, on_change)

idx = 0
prev_point = (0,0,0)
while(True):
    ret, frame = cap.read()
    # frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    # frame = np.split(frame,2, axis = 1)[1]
    val = cv2.getTrackbarPos('Manual Lock', 'frame')
    if val == 1:
        cont.locked = True
    else:
        cont.locked = False

    pt, sorted_contours, clusters = cont.find_contours(frame, 10, offset)
    a,b,r = pt
    plot_img = frame.copy()
    cv2.circle(plot_img, (a, b), r, (0, 255, 0), 2)
    cv2.circle(plot_img, (a, b), 1, (0, 0, 255), 3)
    cv2.circle(plot_img, (a, b), r - offset, (0, 0, 255), 2)
    cv2.drawContours(plot_img, sorted_contours, -1,(0,255, 0),2)
    cv2.drawContours(plot_img, clusters, -1,(0,0,255),2)

    for c in sorted_contours:
        # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # draw the contour and center of the shape on the image
        #cv2.drawContours(plot_img, [c], -1, (0, 255, 0), 2)
        cv2.circle(plot_img, (cX, cY), 2, (0, 0, 255), -1)
        cv2.putText(plot_img, f"{cX},{cY}", (cX - 20, cY - 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # cv2.putText(plot_img, f"{cv2.contourArea(c)}", (cX - 20, cY - 20),
        # cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1)
    #out.write(with_contours)

    for c in clusters:
        # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # draw the contour and center of the shape on the image
        #cv2.drawContours(plot_img, [c], -1, (0, 255, 0), 2)
        cv2.circle(plot_img, (cX, cY), 2, (0, 0, 255), -1)
        # cv2.putText(plot_img, f"{cX},{cY}", (cX - 20, cY - 20),
        # cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)

        # cv2.putText(plot_img, f"{cv2.contourArea(c)}", (cX - 20, cY - 20),
        # cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1)

    if prev_point is pt:
        idx += 1
    else:
        idx = 0
    prev_point = pt


    if idx >= 30:
        message = 'LOCKED'
        color = (0,255,0)
    else:
        message = 'SEARCHING'
        color = (0,0,255)

    cv2.putText(plot_img, "TARGET:", (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)
    cv2.putText(plot_img, f"{message}", (125,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)
    cv2.putText(plot_img, f"Found: {len(sorted_contours)}", (25,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)

    #cv2.setWindowProperty('frame', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow('frame',plot_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
#out.release()
cv2.destroyAllWindows()

centers_cuboids = cont.contour_centers(sorted_contours)

# dash.ClearError()
# dash.EnableRobot()

# utils.default_pos(move)
# for center in centers_cuboids:
#     X, Y, _ = tf_mtx @ (center[0], center[1], 1)
#     move.MovL(X, Y, -70,  0)
#     move.MovL(X, Y, -95,  0)
#     move.MovL(X, Y, -70,  0)
# utils.default_pos(move)

#dash.DisableRobot()

KeyboardInterrupt: 

In [9]:
len(centers_cuboids)

49

In [56]:
dash.DisableRobot()

'0,{},DisableRobot();'

In [12]:
dash.ClearError()
dash.EnableRobot()

'0,{},EnableRobot();'

In [7]:
for center in centers_cuboids:
    X, Y, _ = tf_mtx @ (center[0], center[1], 1)
    move.MovL(X, Y, -70,  0)
    move.MovL(X, Y, -95,  0)
    move.MovL(X, Y, -70,  0)
utils.default_pos(move)

'0,{},JointMovJ(0.000000,0.000000,0.000000,0.000000);'

In [6]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, (1024, 768))

while True:
    ret, frame = cap.read()

    img = frame.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    # blur = cv2.GaussianBlur(thresh, (5,5), 0)
    # kernel = np.ones((4,4), np.uint8)
    # erosion = cv2.erode(blur, kernel, iterations=1)

    # kernel = np.ones((3,3),np.uint8)
    # #closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    # #opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    # dilation = cv2.dilate(thresh,kernel,iterations = 3)

    # blurred = cv2.blur(dilation, (7, 7))
    detected_circles = cv2.HoughCircles(image=gray,
                                        method=cv2.HOUGH_GRADIENT,
                                        dp=1,
                                        minDist=30,
                                        param1=100,
                                        param2=40,
                                        minRadius=1,
                                        maxRadius=30
                                        )

    if detected_circles is not None and detected_circles.shape[1] == 96:
        break
cap.release()
plot_img = frame.copy()
circles = np.uint16(np.around(detected_circles))
for i in circles[0,:]:
    center = (i[0], i[1])

    cv2.circle(plot_img, center, 1, (255, 0 ,0), 3)
    radius = i[2]
    cv2.circle(plot_img, center, radius, (0, 255 ,0), 3)

cv2.imshow('frame', plot_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
well_centers = circles[0][:,:-1]
well_centers = np.array(sorted(well_centers, key = lambda x: x[1]))
well_centers = np.split(well_centers, 12)
well_centers = np.concatenate(np.array([sorted(wells, key = lambda y: y[0]) for wells in well_centers]))

In [9]:
dash.ClearError()
dash.EnableRobot()

for center in well_centers:
    X, Y, _ = tf_mtx @ (center[0], center[1], 1)
    move.MovL(X, Y, -70,  0)
    time.sleep(0.15)
    move.MovL(X, Y, -80,  0)
    time.sleep(0.15)
    move.MovL(X, Y, -70,  0)
    time.sleep(0.15)
utils.default_pos(move)

'0,{},MovL(345.072610,-49.365929,-70.000000,0.000000);'

In [49]:
print(well_centers[48:].shape)

(48, 2)


In [31]:
utils.report_error(dash)


|CODE: 50|
__________________________________________________
Possible reasons from ALARM SERVO:
Invalid velocity or acceleration values.
__________________________________________________
Possible reasons from ALARM CONTROLLER:
Joint3 overspeed.


In [15]:
keys = utils.Keyboard(dash)
keys.execute()
xys = [(arr[0], arr[1]) for arr in keys.coords]

Position saved!
Position saved!
Position saved!
Position saved!
Special key pressed: Key.esc


In [16]:
keys.coords

[array([ 344.984307, -114.147626,  -85.914696,    7.495909]),
 array([ 245.974027, -113.540254,  -82.485573,    1.026354]),
 array([246.632716, -49.664539, -81.510056,  14.440699]),
 array([345.928205, -50.466672, -82.685738,  17.520424])]

In [17]:
robot_coor = utils.assign_corners(xys, reverse=True)

In [18]:
mean_ux = np.mean([robot_coor['ul'][0], robot_coor['ur'][0]])
mean_lx = np.mean([robot_coor['ll'][0], robot_coor['lr'][0]])

mean_uy = np.mean([robot_coor['ur'][1], robot_coor['lr'][1]])
mean_ly = np.mean([robot_coor['ll'][1], robot_coor['ul'][1]])

In [19]:
well_plate_xs = np.linspace(mean_ux, mean_lx, 12)
well_plate_ys = np.linspace(mean_uy, mean_ly, 8)

In [20]:
dash.ClearError()
dash.EnableRobot()

utils.default_pos(move)
for x in well_plate_xs:
    for y in well_plate_ys:
        move.MovL(x, y, -70,  0)
        time.sleep(0.15)
        move.MovL(x, y, -80,  0)
        time.sleep(0.15)
        move.MovL(x, y, -70,  0)
        time.sleep(0.15)

utils.default_pos(move)

'0,{},JointMovJ(0.000000,0.000000,0.000000,0.000000);'

In [21]:
move.JointMovJ(0,0,0,0)

'0,{},JointMovJ(0.000000,0.000000,0.000000,0.000000);'

In [44]:
well_plate_coords = np.concatenate(np.stack(np.meshgrid(well_plate_xs, well_plate_ys), axis = 2))

In [55]:
dash.ClearError()
dash.EnableRobot()

utils.default_pos(move)
for idx, center in enumerate(centers_cuboids):
    X, Y, _ = tf_mtx @ (center[0], center[1], 1)
    move.MovL(X, Y, -70,  0)
    time.sleep(0.5)
    move.MovL(X, Y, -95,  0)
    time.sleep(0.5)
    move.MovL(X, Y, -70,  0)
    time.sleep(0.5)
    
    x,y = well_plate_coords[idx]
    move.MovL(x, y, -70,  0)
    time.sleep(0.5)
    move.MovL(x, y, -90,  0)
    time.sleep(0.5)
    move.MovL(x, y, -70,  0)
    time.sleep(0.5)

utils.default_pos(move)

'0,{},JointMovJ(0.000000,0.000000,0.000000,0.000000);'